# Dataset

1. Data used : Seattle Checkouts by Title (6.62GB After Extraction) [ https://www.kaggle.com/city-of-seattle/seattle-checkouts-by-title ]
2. Description : This dataset includes a monthly count of Seattle Public Library checkouts by title for physical and electronic items. The dataset begins with checkouts that occurred in April 2005.

# Initiating Spark

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("simple data mining").getOrCreate()

In [4]:
#test wheter spark successfully created or not
print(spark)

# Loading Dataset

In [5]:
df = spark.read.csv("D:/Repos/Resource/Lib-Checkout/checkouts-by-title.csv", header=True, inferSchema=True)

In [6]:
#counting rows

df.count()

32723545

In [7]:
#show schema

df.schema

StructType(List(StructField(UsageClass,StringType,true),StructField(CheckoutType,StringType,true),StructField(MaterialType,StringType,true),StructField(CheckoutYear,IntegerType,true),StructField(CheckoutMonth,IntegerType,true),StructField(Checkouts,IntegerType,true),StructField(Title,StringType,true),StructField(Creator,StringType,true),StructField(Subjects,StringType,true),StructField(Publisher,StringType,true),StructField(PublicationYear,StringType,true)))

In [8]:
#creating datas as a temporary SQL View

df.createOrReplaceTempView('libchecksout')

# Simple Data Mining

In [9]:
#Show which material type has the most record

mattype = spark.sql("SELECT MaterialType , SUM(Checkouts) AS TOTAL \
                        FROM libchecksout \
                        GROUP BY MaterialType \
                        ORDER BY TOTAL DESC")

In [10]:
mattype.show()

+--------------------+--------+
|        MaterialType|   TOTAL|
+--------------------+--------+
|                BOOK|54356026|
|           VIDEODISC|28766230|
|           SOUNDDISC|13710478|
|               EBOOK| 8923137|
|           AUDIOBOOK| 3773748|
|           VIDEOCASS| 1524029|
|                SONG| 1311618|
|               MUSIC|  364926|
|            MAGAZINE|  361322|
|           SOUNDCASS|  330665|
|               MIXED|  253790|
|               MOVIE|  212605|
|                  CR|  172233|
|SOUNDDISC, VIDEODISC|  133863|
|              VISUAL|  110539|
|          TELEVISION|   87831|
|            SOUNDREC|   76105|
|               VIDEO|   71701|
|               COMIC|   43928|
|                 KIT|   43737|
+--------------------+--------+
only showing top 20 rows



In [12]:
#show the most top publisher each year

mostpublisher = spark.sql("SELECT CheckoutYear , Publisher, SUM(Checkouts) AS Total \
                            FROM libchecksout \
                            GROUP BY CheckoutYear, Publisher \
                            ORDER BY Total DESC")

In [13]:
mostpublisher.show()

+------------+--------------------+-------+
|CheckoutYear|           Publisher|  Total|
+------------+--------------------+-------+
|        2006|                null|4061261|
|        2008|                null|3971560|
|        2007|                null|3735222|
|        2009|                null|2880721|
|        2010|                null|2528459|
|        2005|                null|2491331|
|        2011|                null|1943663|
|        2012|                null|1313496|
|        2013|                null| 901746|
|        2014|                null| 550253|
|        2015|                null| 468948|
|        2018|  Random House, Inc.| 444055|
|        2016|  Random House, Inc.| 381015|
|        2015|  Random House, Inc.| 380639|
|        2017|  Random House, Inc.| 359447|
|        2016|                null| 334879|
|        2018|HarperCollins Pub...| 323504|
|        2018|Penguin Group (US...| 308035|
|        2014|  Random House, Inc.| 304388|
|        2018|       Books on Ta

In [15]:
#show the most rapid growth of checkouts of the library's properties in 2017-2018

rapidprops = spark.sql("SELECT SUM(Checkouts) as Total , Title \
                        FROM libchecksout \
                        WHERE CheckoutYear BETWEEN 2017 AND 2018 \
                        GROUP BY Title \
                        ORDER BY Total DESC")

In [17]:
rapidprops.show()

+-----+--------------------+
|Total|               Title|
+-----+--------------------+
|11404|In Building Devic...|
|10900|      The New Yorker|
|10296|SPL HotSpot : con...|
| 9868|The hate u give /...|
| 9267|  FlexTech--Laptops.|
| 6543|           Us Weekly|
| 6331|       The Economist|
| 6006|SPL HotSpot : con...|
| 5904|Lion / The Weinst...|
| 5860|Hillbilly Elegy: ...|
| 5718|Educated : a memo...|
| 5711|The woman in the ...|
| 5485|Evicted : poverty...|
| 5234|Arrival / Paramou...|
| 5067|Little Fires Ever...|
| 4946|The Goldfinch: A ...|
| 4942|Manchester by the...|
| 4911|The power : a nov...|
| 4817|Lincoln in the ba...|
| 4801|So you want to ta...|
+-----+--------------------+
only showing top 20 rows



# Export and Simple Data Mining

Let's say i want to see the growth of the record of the first row's Title of the 3rd example each year.
So i need to export it first since i can't see it well on notebook.

In [18]:
#export it into a csv file

rapidprops.coalesce(1).write.csv("C:/Repos/jupyternotes/Resource/most-rapid-growth-in-2017-2018.csv")

In [22]:
# the result is "In Building Device Checkout"
# lets find its growth each year start from the very beginning its checked out.

yearlygrowth = spark.sql("SELECT DISTINCT CheckoutYear,Title, SUM(Checkouts) FROM libchecksout WHERE Title='In Building Device Checkout' GROUP BY CheckoutYear, Title ORDER BY CheckoutYear ASC")

In [23]:
yearlygrowth.show()

+------------+--------------------+--------------+
|CheckoutYear|               Title|sum(Checkouts)|
+------------+--------------------+--------------+
|        2013|In Building Devic...|            12|
|        2014|In Building Devic...|           298|
|        2015|In Building Devic...|           318|
|        2016|In Building Devic...|          2467|
|        2017|In Building Devic...|          5401|
|        2018|In Building Devic...|          6003|
+------------+--------------------+--------------+

